In [ ]:
"""Liquid Argon Dynamics Simulation"""

__authors__ = ["Olaseni Sode"]
__email__   = ["osode@calstatela.edu"]
__date__      = "2024-10-09"

# Liquid Argon Simulation

In this assignment, you will build your own molecular dynamics (MD) simulation engine to model the behavior of a simple molecular system—liquid argon. By constructing the simulation from scratch, you will gain a deeper understanding of how molecular systems are sampled through phase space and how microscopic interactions give rise to macroscopic properties.

Objective: Implement a molecular dynamics simulation that computes the motion of argon atoms using the Lennard-Jones potential, applies periodic boundary conditions, and analyzes the system's properties over time.

### Assignment Overview:
1. Setting Up the Simulation Environment
2. Defining Essential Functions
3. Initializing the Simulation
4. Run the Simulation Loop
5. Visualization of the Simulation
6. Implement a Thermostat (Optional)
7. Compute Radial Distribution Function (Optional)

### 1. Setting up Simulation Environment

#### Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from random import random
from tqdm import tqdm  # For progress bars during simulation

#### Define simulation parameters
Set the fundamental constants and simulation parameters for argon.

In [ ]:
# Constants for Argon
mass = 39.948  # Atomic mass units (amu)
epsilon = 0.996  # Depth of potential well (kJ/mol)
sigma = 0.3405  # Finite distance at which the interparticle potential is zero (nm)

### 2. Defining Essential Functions

#### Define a Lennard-Jones energy function
The Lennard-Jones energy function is used to calculate the potential energy experience between pairs of particles. It is defined as, 

$$U_{ lj} = 4\epsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^6 \right] $$

Implement this equation into the `e_lj` function in the cell below

In [ ]:
def e_lj(r, epsilon, sigma):
    lj = ___________ 
    return lj


#### Force Calculation Function
Implement the function to compute forces analytically using the Lennard-Jones potential. (This function is already completed for you).

In [ ]:
def compute_forces(positions, L, epsilon, sigma):
    N = len(positions)
    forces = np.zeros_like(positions)
    
    for i in range(N):
        for j in range(i + 1, N):
            delta = positions[j] - positions[i]
            # Apply minimum image convention
            delta = delta - L * np.rint(delta / L)
            r = np.sqrt(np.dot(delta, delta))
            
            # Skip if distance is zero (avoid division by zero)
            if r == 0:
                continue
            
            # Compute force magnitude
            r_inv = 1.0 / r
            r_inv6 = (sigma * r_inv) ** 6
            r_inv12 = r_inv6 ** 2
            force_scalar = 24 * epsilon * r_inv * (2 * r_inv12 - r_inv6)
            force_vector = force_scalar * delta / r  # Normalize delta
            
            # Update forces
            forces[i] += force_vector
            forces[j] -= force_vector  # Newton's third law
    return forces


#### Define potential, kinetic and total energy for the system
Let's now write functions that will compute the overall potential energy and kinetic energy for our system. The equations for these terms are given below as, 

$${\rm PE} = \sum_{j>i}^N U_{lj}(i,j) $$

$${\rm KE} = \frac{1}{2}\sum^N_i m_i {\boldsymbol v}_i^2$$

The function for the kinetic energy is already given to you, but you will need to write out a function for the potential energy.

In [ ]:
def kinetic(velocity, mass):
    # Constants for unit conversion
    amu_to_kg = 1.66053906660e-27  # kg per amu
    nmps_to_mps = 1e3  # m/s per nm/ps
    Na = 6.02214076e23  # Avogadro's number
    
    # Convert mass from amu to kg
    mass_kg = mass * amu_to_kg
    
    # Convert velocity from nm/ps to m/s
    velocity_m_s = velocity * nmps_to_mps
    
    # Compute kinetic energy in joules
    ke_joule = 0.5 * mass_kg * np.sum(velocity_m_s ** 2)
    
    # Convert kinetic energy to kJ/mol
    ke_kj_per_mol = (ke_joule * Na) / 1000  # Divide by 1000 to convert J to kJ
    
    return ke_kj_per_mol


For the potential energy expression don't forget about the minimum image convention for periodic boundary conditions.

**What is the Minimum Image Convention?**
- The minimum image convention ensures that when calculatinginteractions between particles in a periodic system, each particle only interacts with the closest image of every other particle. This means that even though particles have infinite periodic images due to the boundary conditions, we consider only the nearest one for each pairwise interaction.

**Why is it Important?**
- *Physical Accuracy:* Without applying the minimum image convention, particles might interact with distant images of other particles, leading to unrealistic representations of the system. The convention ensures that the simulation mimics an infinite system while keeping interactions physically meaningful.

- *Computational Efficiency:* It reduces computational load by limiting the number of interactions that need to be calculated, as only the nearest images are considered.

In [ ]:
def potential(positions, L, eps, sig):
    """
    Calculate the total potential energy of the system using the Lennard-Jones potential
    and applying the minimum image convention to account for periodic boundary conditions.

    Parameters:
    positions -- (N, 3) NumPy array containing the positions of N particles
                 Each row corresponds to the x, y, z coordinates of a particle.
    L         -- Length of the simulation box (assuming a cubic box)
    eps       -- Depth of the Lennard-Jones potential well (epsilon)
    sig       -- Finite distance at which the interparticle potential is zero (sigma)

    Returns:
    energy    -- Total potential energy of the system
    """
    
    # Initialize the total potential energy to zero
    energy = 0.0
    
    # number of atoms
    N = len(coordinate)
    
    for i in range(1,N):
        for j in range(i+1,N):

            # Compute the displacement vector between atoms i and j
            dvect = ___________ 
            
            # Apply the minimum image convention to the displacement vector
            # This adjusts the vector to account for periodic boundary conditions,
            # ensuring that we consider the shortest possible distance between particles
            # find nearest image of atom pair distance 
            dvect = ___________ 
            
            # Compute the distance between atoms i and j using the adjusted displacement vector
            d = ___________
            
            # Calculate the Lennard-Jones potential energy for this pair of atoms
            # e_lj is the Lennard-Jones potential function, which depends on the distance d,
            # and the Lennard-Jones parameters epsilon (eps) and sigma (sig)
            energy += e_lj(d, eps, sig)
    
    # Return the total potential energy of the system
    return energy

#### Temperature Calculation Function
Implement the function to calculate the temperature of the system. (This function is already completed for you).

In [ ]:
import numpy as np

def temperature(velocity, mass):
    """
    Calculate the temperature of a system of particles based on their velocities
    using the equipartition theorem of kinetic energy.

    Parameters:
    velocity -- (N, 3) NumPy array containing the velocities of N particles
                Each row corresponds to the velocity in x, y, z directions for a particle (in nm/ps).
    mass     -- Mass of the particles (in atomic mass units, amu)

    Returns:
    T        -- Temperature of the system in Kelvin (K)
    """
    # Number of particles in the system (N)
    N = len(velocity)

    # Constants
    k_B = 1.380649e-23  # Boltzmann constant in Joules per Kelvin (J/K)
    amu_to_kg = 1.66053906660e-27  # Conversion factor from amu to kilograms (kg/amu)
    nmps_to_mps = 1e3  # Conversion factor from nm/ps to meters per second (m/s)

    # 1. **Convert mass to kilograms**:
    # Mass of the particles is given in atomic mass units (amu).
    # Convert it to SI units (kg) for the energy calculations.
    mass_kg = mass * amu_to_kg  # kg

    # 2. **Convert velocities to meters per second (m/s)**:
    # The velocities are initially in nanometers per picosecond (nm/ps).
    # Convert them to meters per second (m/s), which is the standard unit for velocity.
    velocity_m_s = velocity * nmps_to_mps  # m/s

    # 3. **Calculate the total kinetic energy**:
    # Kinetic energy is given by the formula: KE = 0.5 * m * v^2.
    # Here, we sum the squared velocities of all particles, multiply by the mass,
    # and calculate the total kinetic energy of the system.
    ke_joule = 0.5 * mass_kg * np.sum(velocity_m_s ** 2)  # Joules (J)

    # 4. **Compute the temperature**:
    # According to the equipartition theorem, the temperature of the system is related to
    # the kinetic energy by the formula: KE = (3/2) * N * k_B * T.
    # Rearranging this formula, we can solve for the temperature: T = (2 * KE) / (3 * N * k_B).
    T = (2 * ke_joule) / (3 * N * k_B)  # Temperature in Kelvin (K)

    # 5. **Return the computed temperature**:
    return T


### 3. Initialize the Simulation

#### Set Simulation Parameters
Define the number of atoms, the size of the simulation box, and the time step. Try initially 20 atoms, a box length of 5 nm, a time step of 0.005 ps, and 5000 simulation steps.

In [ ]:
N = ___________   # Number of atoms
L = ___________   # Box length in nm
dt = ___________   # Time step in ps
nsteps = ___________   # Number of simulation steps


#### Initialize Positions
The `get_sphere_distribution` function generates a set of n random points (particle positions) within a box of specified dimensions Ls, ensuring that no two points are closer than a specified minimum distance dmin. This helps in initializing a molecular dynamics simulation with particles that are well-distributed and avoids overlaps that could cause unrealistic forces.

In [ ]:
def get_sphere_distribution(n, dmin, Ls, maxiter=1e4, allow_wall=True):
    """Get random points in a box with given dimensions and minimum separation.
    
    Parameters:
      
    - n: number of points
    - dmin: minimum distance
    - Ls: dimensions of box, shape (3,) array 
    - maxiter: maximum number of iterations.
    - allow_wall: whether to allow points on wall; 
       (if False: points need to keep distance dmin/2 from the walls.)
        
    Return:
        
    - ps: array (n, 3) of point positions, 
      with 0 <= ps[:, i] < Ls[i]
    - n_iter: number of iterations
    - dratio: average nearest-neighbor distance, divided by dmin.
    
    Note: with a fill density (sphere volume divided by box volume) above about
    0.53, it takes very long. (Random close-packed spheres have a fill density
    of 0.64).
    
    Author: Han-Kwang Nienhuys (2020)
    Copying: BSD, GPL, LGPL, CC-BY, CC-BY-SA
    See Stackoverflow: https://stackoverflow.com/a/62895898/6228891 
    """
    Ls = np.array(Ls).reshape(3)
    if not allow_wall:
        Ls -= dmin
    
    # filling factor; 0.64 is for random close-packed spheres
    # This is an estimate because close packing is complicated near the walls.
    # It doesn't work well for small L/dmin ratios.
    sphere_vol = np.pi/6*dmin**3
    box_vol = np.prod(Ls + 0.5*dmin)
    fill_dens = n*sphere_vol/box_vol
    if fill_dens > 0.64:
        msg = f'Too many to fit in the volume, density {fill_dens:.3g}>0.64'
        raise ValueError(msg)
    
    # initial try   
    ps = np.random.uniform(size=(n, 3)) * Ls
    
    # distance-squared matrix (diagonal is self-distance, don't count)
    dsq = ((ps - ps.reshape(n, 1, 3))**2).sum(axis=2)
    dsq[np.arange(n), np.arange(n)] = np.infty

    for iter_no in range(int(maxiter)):
        # find points that have too close neighbors
        close_counts = np.sum(dsq < dmin**2, axis=1)  # shape (n,)
        n_close = np.count_nonzero(close_counts)
        if n_close == 0:
            break
        
        # Move the one with the largest number of too-close neighbors
        imv = np.argmax(close_counts)
        
        # new positions
        newp = np.random.uniform(size=3)*Ls
        ps[imv]= newp
        
        # update distance matrix
        new_dsq_row = ((ps - newp.reshape(1, 3))**2).sum(axis=-1)
        dsq[imv, :] = dsq[:, imv] = new_dsq_row
        dsq[imv, imv] = np.inf
    else:
        raise RuntimeError(f'Failed after {iter_no+1} iterations.')

    if not allow_wall:
        ps += dmin/2
    
    dratio = (np.sqrt(dsq.min(axis=1))/dmin).mean()
    return ps, iter_no+1, dratio


positions, n_iter, dration = get_sphere_distribution(N,1.0,(L,L,L))

#### Initialize Velocities
Set initial velocities close to zero or sample from a Maxwell-Boltzmann distribution.

In [ ]:
def set_velocities_near_zero(N, magnitude=1e-5):
    velocities = np.random.uniform(-magnitude, magnitude, (N, 3))
    return velocities

velocities = set_velocities_near_zero(N)

#### Remove Net Momentum
Ensure that the system has no net momentum.

In [ ]:
def remove_net_momentum(velocities, mass):
    total_mass = N * mass
    momentum = np.sum(velocities * mass, axis=0)
    velocity_correction = momentum / total_mass
    velocities -= velocity_correction
    return velocities

velocities = remove_net_momentum(velocities, mass)

#### Compute Initial Forces
Calculate the initial forces acting on the particles.

In [ ]:
forces = compute_forces(positions, L, epsilon, sigma)

### 4. Implement Velocity Verlet Algorithm and Run the Simulation Loop

To propagate our system forward in time, we use the Velocity Verlet algorithm, which updates both the positions and velocities of the particles at each simulation step. The Velocity Verlet algorithm is widely used in molecular dynamics simulations due to its simplicity, stability, and good energy conservation properties.

The Velocity Verlet algorithm consists of the following steps:
1. **Position Update**
$${\bf r}^{n+1} = {\bf r}^{n} + {\bf v}^{n}\Delta t + {\bf a}^n\frac{\Delta t^2}{2},      \qquad(1)  $$

2. **Force and Acceration Calculation**
- Compute the forces ${\bf F}^{n+1}$ acting on the particles at the new positions ${\bf r}^{n+1}$.
- Update the accelerations: 
$${\bf a}^{n+1} = \frac{{\bf F}^{n+1}}{m},      \qquad(2)  $$

3. **Velocity Update**
$${\bf v}^{n+1} = {\bf v}^{n} + \frac{1}{2} ({\bf a}^{n} + {\bf a}^{n+1} ) \Delta t.      \qquad(3)  $$


In [ ]:
# Arrays to store energy and temperature for analysis
energies = []
temperatures = []

print("Starting simulation...")
for step in tqdm(range(nsteps)):
    # **Update positions using the Velocity Verlet algorithm**
    positions = ____________  # TODO: Implement position update here
    
    # **Apply periodic boundary conditions to positions**
    positions = ____________  # TODO: Apply periodic boundary conditions to positions
    
    # **Store current forces before computing new ones**
    forces_old = ____________  # TODO: Store the current forces
    
    # **Compute new forces based on updated positions**
    forces = ____________  # TODO: Compute new forces using compute_forces function
    
    # **Update velocities using the Velocity Verlet algorithm**
    velocities = ____________  # TODO: Implement velocity update here
    
    # **Calculate kinetic and potential energies**
    ke = ____________  # TODO: Implement potential energy calculation using potential function
    pe = ____________  # TODO: Implement potential energy calculation using potential function
    total_energy = ke + pe
    energies.append(total_energy)
    
    # **Calculate temperature of the system**
    temp = ____________  # TODO: Implement temperature calculation using temperature function
    temperatures.append(temp)
    
    
print("Simulation completed.")

**Question: <span style="color:red">Explain how the Lennard-Jones potential leads to both attractive and repulsive forces between argon atoms.</span>**

    
**Answer:** 

**Question: <span style="color:red">Why are periodic boundary conditions used in molecular dynamics simulations, and how does the minimum image convention help implement them?</span>**

**Answer:**

**Question: <span style="color:red">After running the simulation, plot the total energy over time. Is energy conserved in your simulation? If not, what factors might be causing energy drift?</span>**

**Answer:** 

In [ ]:
# Plotting total energy over time
plt.figure(figsize=(10, 6))
plt.plot(energies)
plt.title('Total Energy Over Time')
plt.xlabel('Simulation Step')
plt.ylabel('Energy (kJ/mol)')
plt.show()


**Question: <span style="color:red">Plot the temperature over time. Does the system maintain the desired temperature? What could be done to control the temperature more effectively?</span>**

**Answer:**

In [ ]:
# Plotting temperature over time
plt.figure(figsize=(10, 6))
plt.plot(temperatures)
plt.title('Temperature Over Time')
plt.xlabel('Simulation Step')
plt.ylabel('Temperature (K)')
plt.show()

**Question: <span style="color:red">Experiment with different time step sizes (e.g., 0.001 ps, 0.005 ps, 0.01 ps). How does the time step affect the accuracy and stability of your simulation?</span>**

**Answer:**

### 5. Visualization of the Simulation with NGLView
Now that you've run your simulation and collected the trajectory data, you can visualize the motion of the argon atoms using `nglview`. This will help you observe the behavior of the system over time.

#### Preparing the Trajectory File
During the simulation, you need to save the positions of the particles at each time step in a format that MDAnalysis and nglview can read. The XYZ format is commonly used for this purpose.

#### Modify the Simulation Loop to Save Trajectory
Add a function to write the positions to an XYZ file:

In [ ]:
def write_xyz(positions, step, filename='argon_simulation.xyz'):
    with open(filename, 'a') as f:
        f.write(f"{len(positions)}\n")
        f.write(f"Step {step}\n")
        for pos in positions:
            f.write(f"Ar {pos[0]} {pos[1]} {pos[2]}\n")

Clear the file before starting the simulation with the following command:

```
# Before starting the loop, clear or create the file
with open('argon_simulation.xyz', 'w') as f:
    f.write('')
```    

#### Update Simulation Loop
In the cell below, copy the simulation loop you created from before and update it with a call to `write_xyz`.

In [ ]:
# Add in your new simulation loop here.
# Make sure to reinitialize your velocities
# and forces.
# Include your write_xyz function call during
# the simulation loop and don't forget to
# clear and create the file before the loop.


Now that you have the trajectory saved in argon_simulation.xyz, you can load it using MDAnalysis and visualize it with nglview.


In [ ]:
import MDAnalysis as mda
import nglview as nv

# load the argon simulation trajectory
a = mda.Universe("argon_simulation.xyz", format='XYZ')

# specify the dimension
# and angles of unit cell
a.dimensions = [L, L, L, 90, 90, 90]

# load system into nglview
# and change the representations
view = nv.show_mdanalysis(a)
view.remove_ball_and_stick()
view.center()
view.add_unitcell()
view.add_representation('spacefill',radius='1')

view

**Question: <span style="color:red">Observe how particles behave when they reach the boundaries of the simulation box. Do they appear on the opposite side as expected due to periodic boundary conditions?</span>**

**Answer:** 

.

**Question: <span style="color:red">Describe any patterns or behaviors you notice in the motion of the argon atoms. Are there any indications of clustering or phase changes?</span>**

**Answer:** 

.

**Question: <span style="color:red">What modifications could you make to your simulation to observe liquid or solid behavior in argon?</span>**

**Answer:** 

.

### 6. Implement a Thermostat (Optional Exercise)
Implement a simple thermostat, such as the Berendsen thermostat, to control the temperature of your system.

#### Berendsen Thermostat 
In molecular dynamics simulations, controlling the temperature of the system is crucial, especially when simulating processes at constant temperature (canonical ensemble). The Berendsen thermostat is a widely used method for temperature control due to its simplicity and ease of implementation. It gently adjusts the system's temperature towards a desired value by rescaling particle velocities, simulating a weak coupling to an external heat bath.

The Berendsen thermostat modifies the velocities of particles to bring the system's temperature $T$ closer to a target temperature $T_0$. It does this by scaling the velocities at each time step using a scaling factor $\lambda$:
$${\bf v}^{n+1} = \lambda{\bf v}^{n} .      \qquad(4)  $$

The scaling factor $\lambda$ is calculated based on the current temeprature, the desired temperature, and the coupling parameter: 

$$\lambda = \sqrt{1+\frac{\Delta t}{\tau_T}\left (\frac{T_0}{T}-1\right )} \qquad(5)  $$
where $\tau_T$ is a thermostat time constant (relaxation time).

Implement the berendsen thermostat below:

In [ ]:
def apply_berendsen_thermostat(velocities, mass, desired_temperature, tau, dt):
    """
    Apply the Berendsen thermostat to adjust velocities.

    Parameters:
    velocities          -- (N, 3) array of particle velocities in nm/ps
    mass                -- Mass of a particle in atomic mass units (amu)
    desired_temperature -- Target temperature T0 in Kelvin
    tau                 -- Thermostat time constant in picoseconds (ps)
    dt                  -- Time step size in picoseconds (ps)

    Returns:
    velocities_scaled   -- Adjusted velocities after applying the thermostat
    """
    # **Calculate the current temperature**
    current_temperature = ____________  # TODO: Use your temperature function to compute current temperature
    
    # **Calculate the scaling factor lambda**
    lambda_factor = ____________  # TODO: Compute the scaling factor using the Berendsen thermostat formula
    
    # **Rescale velocities**
    velocities_scaled = ____________  # TODO: Rescale velocities using the scaling factor
    
    # **Return the scaled velocities**
    return velocities_scaled

#### Define desired temperature and thermostat parameters
Before entering the simulation loop, specify the target temperature and the thermostat coupling constant. Also, ensure that you have defined the time step `dt`.

In [ ]:
# Define the desired temperature (in Kelvin)
desired_temperature = 100  # Adjust as needed

# Define the time step (in picoseconds)
dt = 0.005  # Example time step

# Define the thermostat time constant (coupling constant)
tau_T = 100 * dt  # Adjust as needed (e.g., tau_T = 0.5 ps)

#### Modify the Simulation Loop to Apply the Thermostat
Inside your simulation loop, apply the Berendsen thermostat at regular intervals. For example, you can apply it every 100 steps.

```
# Apply Berendsen thermostat every 100 steps
if step % 100 == 0:
    velocities = apply_berendsen_thermostat(velocities, mass, desired_temperature, tau_T, dt)
```

In [ ]:
# Add in your new simulation loop here.
# Make sure to reinitialize your velocities
# and forces.
# Apply the Berendsen thermostat every 100 
# steps with the code above in the simulation
# loop.

**Question: <span style="color:red">Run simulations at different temperatures and visualize the trajectories. How does temperature affect the motion of the atoms?</span>**

**Answer:** 

.

**Question: <span style="color:red">How does the frequency at which you apply the thermostat (e.g., every 10 steps vs. every 100 steps) influence the temperature control and system dynamics?</span>**

**Answer:** 

.

**Question: <span style="color:red">Does the total energy of the system remain constant when using the Berendsen thermostat? Why or why not?</span>**

**Answer:** 

.

### 7. Analyze Radial Distribution Function (Optional Exercise)

One powerful tool for analyzing such structural properties is the Radial Distribution Function (RDF), denoted as 
$g(r)$. The RDF provides statistical information about how particle density varies as a function of distance from a reference particle. Essentially, it answers the question: What is the probability of finding a particle at a distance $r$ from another particle compared to an ideal gas at the same density?

$$ g(r) = \frac{\left <\rho(r)\right>}{\rho_0} $$

where $\left <\rho(r)\right>$ is the average local density at distance $r$, and $\rho_0$ is the average density of the system. 

The RDF is important for phase identification and structural insight. The RDF reveals how particles are spatially arranged, indicating the presence of short-range order, clustering, or layering in the system. Different phases (gas, liquid, solid) exhibit characteristic RDF patterns. For example: the RDF for a gas is close to 1 for all $r$ indicating a random distribution, while the RDF for a solid shows sharp peaks of specific distances due to long-range order.

In [ ]:
bins = 50

In [ ]:
def compute_rdf(positions, L, bins=50, r_max=L/2):
    """
    Compute the radial distribution function g(r) for a set of particle positions.

    Parameters:
    positions -- (N, 3) array of particle positions
    L         -- Simulation box length (assuming cubic box)
    bins      -- Number of bins for the histogram
    r_max     -- Maximum distance to consider (default is L/2)

    Returns:
    r         -- Array of distance values corresponding to g(r)
    g_r       -- Radial distribution function values
    """
    N = len(positions)
    if r_max is None:
        r_max = L / 2.0  # Maximum distance is half the box length due to periodic boundaries

    dr = r_max / bins  # Bin width
    rdf = np.zeros(bins)
    distances = []

    # Compute pairwise distances with minimum image convention
    for i in range(N):
        for j in range(i + 1, N):
            delta = positions[j] - positions[i]
            # Apply minimum image convention
            delta -= L * np.rint(delta / L)
            r = np.sqrt(np.dot(delta, delta))
            if r < r_max:
                bin_index = int(r / dr)
                rdf[bin_index] += 2  # Count pairs twice (i-j and j-i)

    # Normalize RDF
    r = np.linspace(dr / 2, r_max - dr / 2, bins)
    shell_volume = (4 / 3) * np.pi * ((r + dr / 2) ** 3 - (r - dr / 2) ** 3)
    number_density = N / L ** 3
    ideal_gas = number_density * shell_volume * N
    g_r = rdf / ideal_gas

    return r, g_r

Add the following code within the simulation loop to compute the rdf throughout the simulation.
```
        r, g_r = compute_rdf(positions, L, bins, r_max)
        total_rdf += g_r
```

In [ ]:
total_rdf = np.zeros(bins)
# Add in your new simulation loop here.
# Make sure to reinitialize your velocities
# and forces.
# Apply compute_rdf code here. 

In [ ]:
# Compute average of g(r)
g_r_avg = total_rdf / nsteps

In [ ]:
# Plotting the RDF
plt.figure(figsize=(10, 6))
plt.plot(r, g_r_avg)
plt.title('Radial Distribution Function g(r)')
plt.xlabel('Distance r (nm)')
plt.ylabel('g(r)')
plt.grid(True)
plt.show()
